# Material Balances XIV

This lecture continues the discussion of reactor design for systems involving multiple reactions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

## Example Problem 01: Revisiting Benzene Pyrolysis in a PFR

You are carrying out Benzene Pyrolysis in a Plug Flow Reactor operating at 1033K and 1.0 atm.  The two reactions occurring in this system are benzene coupling to form diphenyl and hydrogen followed by a secondary reaction between benzene and diphenyl to form triphenyl and hydrogen:

\begin{align*}
&2B \longleftrightarrow D + H_2\\
&B + D \longleftrightarrow T + H_2
\end{align*}

Both reactions are reversible and ***follow elementary rate laws***.  Rate constants and equilibrium concentration ratios ($K_C$) are given below.

\begin{align*}
&k_1 = 7.0 \ \times 10^5 \ \textrm{L} \ \textrm{mol}^{-1} \ \textrm{h}^{-1}\\
&k_2 = 4.0 \ \times 10^5 \ \textrm{L} \ \textrm{mol}^{-1} \ \textrm{h}^{-1}\\
&K_{C_1} = 0.31\\
&K_{C_2} = 0.48
\end{align*}

If pure benzene is fed into the reactor at 60,000 moles per hour, find the PFR volume required to achieve 50\% conversion of Benzene.  Also, plot the mole fraction of each species as a function of PFR volume.

**Answer**: 403.3 L

### Solution to Example Problem 01

*This is exactly the same as the solution from ***587-L22***, we are just going to expand consideration of the PFR to include yield and selectivity.*

Even though there are multiple reactions, we approach this problem the same way as usual: we write a material balance on Benzene.

$$\frac{dF_B}{dV} = R_B$$

We generally know that $R_B$ is going to be a complex function of concentrations of all of the species present in this system.  That means we can't solve the above balance on Benzene without also solving balances on Diphenyl, Hydrogen, and Triphenyl *at the same time*, i.e., we have to solve the coupled system of differential equations below:

\begin{align}
    \frac{dF_B}{dV} &= R_B \\
    \frac{dF_D}{dV} &= R_D \\
    \frac{dF_H}{dV} &= R_H \\
    \frac{dF_T}{dV} &= R_T \\
\end{align}

So, we've got 4 differential equations that tell us how 4 dependent variables ($F_B$, $F_D$, $F_H$, $F_T$) change as a function of the one independent variable, $V$.  We can solve this system *if* we can define everything on the right hand sides of the above equations ($R_B$, $R_D$, $R_H$, and $R_T$) in terms of $F_B$, $F_D$, $F_H$, $F_T$, and/or $V$.

We know how to do this!!

Generally:

$$R_j = \sum_i \nu_{i,j} r_i$$

So:

\begin{align}
    R_B &= -2r1 - r2 \\
    R_D &=  r1 - r2 \\
    R_H &=  r2 + r2 \\
    R_T &=  r2 \\
\end{align}

We next define reaction rates:

\begin{align}
    r_1 &= k_{1,f}C_B^2 - k_{1,r}C_HC_D \\
    r_2 &= k_{2,f}C_BC_D - k_{2,r}C_HC_T \\
\end{align}

We define concentrations in terms of molar flowrates:

$$C_j = \frac{F_j}{Q}$$

Which throws a volumetric flowrate, $Q$, into the mix.  Fortunately, this is a gas phase reaction at low pressure, so we know we can define $Q$ in terms of the total molar flowrate:

$$Q = \frac{F_{tot}RT}{P}$$

Where

$$F_{tot} = \sum_j F_j$$

With that, we've defined out system of ODES fully as a function of molar flowrates and reactor volume.  We can solve this numerically using solve_ivp.  See below


In [ ]:
###########################################
# Global constants from problem statement #
###########################################

T   = 1033    #K
P   = 1.0     #atm
R   = 0.08206 #L*atm/mol/K
k1f = 7.0e5   #L/mol/h
k2f = 4.0e5   #L/mol/h
KC1 = 0.31
KC2 = 0.48
k1r = k1f/KC1 #L/mol/h
k2r = k2f/KC2 #L/mol/h

# Feed molar flowrates
FBf     = 60000 #mol/h
FDf     = 0
FHf     = 0
FTf     = 0

In [ ]:
def P01(vol, var):
    
    #Dependent variables are all in var
    FB, FD, FH, FT = var
      
    #total molar flowrate, function of individual molar flowrates
    FTOT = FB + FD + FH + FT
    
    #volumetric flowrate
    Q    = FTOT*R*T/P
    
    #Define concentrations
    CB   = FB/Q
    CD   = FD/Q
    CH   = FH/Q
    CT   = FT/Q
    
    #now that we have concentrations, we define reaction rates
    r1   = k1f*CB**2 - k1r*CD*CH
    r2   = k2f*CB*CD - k2r*CT*CH
    
    #With reaction rates, you can define production rates
    RB   = -2*r1 - r2
    RD   =    r1 - r2
    RH   =    r1 + r2
    RT   =         r2
    
    #For a PFR, dFj/dV = Rj, so these are our derivatives of FB, FD, FH, FT
    D1   = RB
    D2   = RD
    D3   = RH
    D4   = RT
    
    #return derivatives of each dependent variable w.r.t. volume
    return [D1, D2, D3, D4]

In [ ]:
#Solve the problem
vspan   = (0, 5000)
var0    = [FBf, FDf, FHf, FTf]
solP01 = solve_ivp(P01, vspan, var0)#, atol = 1e-12, rtol = 1e-12)

#Extract data from the solution structure.

#Volumes from the ODE solver
Vout    = solP01.t

#Molar flowrates as a function of volume.
FBout   = solP01.y[0]
FDout   = solP01.y[1]
FHout   = solP01.y[2]
FTout   = solP01.y[3]

#Sum things up to get the total molar flowrate as a function of volume.
FTot_o  = FBout + FDout + FHout + FTout

#Calculate mole fractions as a function of volume.
yBout   = FBout/FTot_o
yDout   = FDout/FTot_o
yHout   = FHout/FTot_o
yTout   = FTout/FTot_o

#Calculate conversion
XBout   = (FBf - FBout)/FBf

#Plot conversion vs. reactor volume.
plt.figure(figsize = (5, 5))
plt.title('Benzene Conversion vs. PFR Volume', fontsize = 14)
plt.plot(Vout, XBout)
plt.xlim(0, max(vspan))
plt.ylim(0, 0.6)
plt.hlines(0.5, 0, max(vspan), linestyle = 'dashed', color = 'black', linewidth = 1)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('Benzene Conversion', fontsize = 14)
plt.show()

### Consider Maximixing Diphenyl Production as opposed to Benzene Conversion

We already know (from **587-L22**) that we need a 403L PFR in order to achieve 50% conversion of Benzene.  A different question might be what volume *should* the reactor be.  As we discussed in class, that depends on our goal.  If it is to maximize Benzene conversion, then we would pick something like a 3000L reactor, where we reach the maximum allowable conversion of Benzene. In contrast, if our goal is to maximize the production of a certain compound, like diphenyl, we'd make a different choice. In fact, if we really wanted to maximize Diphenyl production, we'd probably want to stick with a reactor volume of about 400L because that is where the mole fraction of diphenyl is at a maximum.  This brings us to a discussion about the need for ***optimization*** in reactor design. This is the third type of numerical method we will use in this course (the first one being ***algebraic equation solvers*** like those in `opt.root()` or `opt.newton()` and the second one being ***ODE/Initial Value Problem solvers*** like those in `solve_ivp()`). There is a brief introduction to numerical methods for optimization (minimization) in **587-R08**, which considers volume and cost minimization by using a series of CSTRs instead of a single CSTR. 

In [ ]:
#Plot mole fractions vs. volume
plt.figure(2, figsize = (5, 5))
plt.title('Species Mole Fractions vs. PFR Volume', fontsize = 14)
plt.plot(Vout, yBout, label = 'yB')
plt.plot(Vout, yDout, label = 'yD')
plt.plot(Vout, yHout, label = 'yH')
plt.plot(Vout, yTout, label = 'yT')
plt.xlim(0, max(vspan))
plt.ylim(0, 1)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('mole fraction', fontsize = 14)
plt.legend()
plt.show(2)

#Plot mole fractions vs. volume
plt.figure(3, figsize = (5, 5))
plt.title('Finding the maximum value for yD', fontsize = 14)
plt.plot(Vout, yBout, label = 'yB')
plt.plot(Vout, yDout, label = 'yD')
plt.plot(Vout, yHout, label = 'yH')
plt.plot(Vout, yTout, label = 'yT')
plt.xlim(0, 800)
plt.ylim(0, 0.25)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('mole fraction', fontsize = 14)
plt.legend()
plt.show(3)

In [ ]:
yDmax1 = max(yDout)
print(yDmax1)
print(f'The maximum diphenyl mole fraction is {yDmax1:0.3f}. \n')

indexmax = np.argmax(yDout)
print(indexmax)
print(yDout[indexmax])
yDmax2   = yDout[indexmax]
Vmax2    = Vout[indexmax]
print(f'The maximum diphenyl mole fraction is {yDmax2:0.3f}, which we obtain at a PFR volume of {Vmax2:0.0f}L.')

### Introducing Yield and Selectivity

The benzene pyrolysis examples highlight the fact that conversion alone is not an adequate process specification when we're dealing with multiple reactions.  We also need a way to quantify the amounts of each product that we make.  This is done using two metrics: ***Yield*** and ***Selectivity***.

We define the ***yield*** as the percentage of our reactant that was converted into a specific product.  

We define the ***overall selectivity*** as the percentage of consumed reactant that was converted into a specific product.

#### Consider A generic pair of reactions

Let's discuss two reactions.  The first one converts our reactant, $A$, into a desired product (valuable, non-toxic, beneficial to society somehow, etc).  The second one occurs in parallel and converts our reactant $A$ into something undesirable (toxic, costly to dispose, greenhouse gas, etc).  We would like to establish a general way for discussing the amounts of $D$ and $U$ that are produced in reaction.

\begin{align}
    (1) \qquad \nu_{A1} \, A \longrightarrow \nu_{D1} D \\
    (2) \qquad \nu_{A2} \, A \longrightarrow \nu_{D1} U \\
\end{align}

##### Yield

Yield is defined as the percentage of our reactant, $A$, that is converted into each product.  So, for a batch reactor (where we usually work with moles of species), the "yield of $D$ with respect to $A$" would be conceptually defined as: the number of moles of $A$ converted into species $D$ divided by the initial number of moles of $A$ put into the system.  In equation form, that looks like:

***In a batch reactor***

$$Y_{D/A} = \frac{\left|\frac{\nu_{A1}}{\nu_{D1}}\right|N_D}{N_{A0}}$$

Similarly, we would define the yield of $U$ with respect to $A$ as the number of moles of $A$ converted into species $U$ divided by the initial number of moles of $A$ put into the system, i.e.:

$$Y_{U/A} = \frac{\left|\frac{\nu_{A2}}{\nu_{U2}}\right|N_U}{N_{A0}}$$

***In a Flow Reactor***

If we were working with a flow reactor, it usually makes more sense to work with molar flowrates instead of number of moles, so we would define the two yields as:

$$Y_{D/A} = \frac{\left|\frac{\nu_{A1}}{\nu_{D1}}\right|F_D}{F_{Af}}$$

and

$$Y_{U/A} = \frac{\left|\frac{\nu_{A2}}{\nu_{U2}}\right|F_U}{F_{Af}}$$

##### Selectivity

For now, we'll only work with a quantity that we call the "Overall Selectivity," which is defined as the percentage of the ***consumed reactant*** that was converted into a specific product.  So, at a conceptual level, if I wanted to define the "overall selectivity of $D$ with respect to $A$," that would be defined as the number of moles of $A$ that were converted to make $D$ divided by the total number of moles of $A$ consumed.  In Equation form:

***In a Batch Reactor***

$$S_{D/A} = \frac{\left|\frac{\nu_{A1}}{\nu_{D1}}\right|N_D}{N_{A0} - N_A}$$

Similarly, the "overall selectivity of $U$ with respect to $A$" is given by the number of moles of $A$ converted to make $U$ divided by the total number of moles of $A$ consumed.  

$$S_{U/A} = \frac{\left|\frac{\nu_{A2}}{\nu_{U2}}\right|N_U}{N_{A0} - N_A}$$

***In a Flow Reactor***

If I'm working with a flow process, I will define these quantities in terms of molar flowrates.

$$S_{D/A} = \frac{\left|\frac{\nu_{A1}}{\nu_{D1}}\right|F_D}{F_{Af} - F_A}$$

and

$$S_{U/A} = \frac{\left|\frac{\nu_{A2}}{\nu_{U2}}\right|F_U}{F_{Af} - F_A}$$

##### Fractional Conversion

These definitions for Yield and Selectivity complement the one for fractional conversion:

**Batch Reactor**

$$X_A = \frac{N_{A0} - N_A}{N_{A0}}$$

**Flow Reactor**

$$X_A = \frac{F_{Af} - F_A}{F_{Af}}$$

As a final note:

1. With the definitions above, fractional conversion, selectivity, and yield will always be between 0 and 1.
2. The yield of a species is equal to the selectivity of that species multiplied by the fractional conversion of the reactant that formed it.

$$Y_{D/A} = S_{D/A}X_A$$

## Example Problem 02: Benzene Pyrolysis with Yields and Selectivities

Re-solve Example Problem 01. Plot yields and overall selectivities to diphenyl and triphenyl with respect to benzene as a function of volume. What PFR volume should you use to maximize the yield of diphenyl?

\begin{align*}
&2B \longleftrightarrow D + H_2\\
&B + D \longleftrightarrow T + H_2
\end{align*}

.

### Solution to Example Problem 02

We solve this essentially the same way as we did Example Problem 01 -- we write the material balances for all species in the reactor, and we solve the resultant coupled system of ODEs using `solve_ivp()`. This part is identical to the solution from **587-L22**.

We recognize that solution of the problem using `solve_ivp()` will give us the molar flowrates of Benzene, Diphenyl, Triphenyl, and Hydrogen as a function of reactor volume.  This is enough information for us to calculate yields and selectivity as a function of reactor volume since both of those quantities are defined in terms of molar flowrates for flow reactors.  Specifically:

**Yield of Diphenyl with respect to Benzene**

According to the first reaction, each diphenyl produced will consume two benzenes, so the correct definition for yield according to the equations in the preceding section is:

$$Y_{D/B} = \frac{2F_D}{F_{Bf}}$$

The coefficients are less clear for triphenyl since each triphenyl consumes one diphenyl...which consumes two benzenes.  The easiest way to handle the yield definition for a sequential reaction like this is to add the overall reactions so that we can relate triphenyl directly to benzene:

\begin{align}
   &2B \leftrightarrow D + H \\
   &B + D \leftrightarrow T + H \\
        \hline
    &3B \leftrightarrow T + 2H \\
\end{align}

From that overall equation, it becomes clear that it takes 3 benzenes to make one triphenyl, so the yield definition is:

$$Y_{T/B} = \frac{3F_T}{F_{Bf}}$$

We extend these ideas to the selectivity definitions, which quantify the percentage of consumed reactant that went to produce diphenyl or triphenyl.

Selectivity to diphenyl with respect to benzene:

$$S_{D/B} = \frac{2F_D}{F_{Bf} - F_B}$$

Selectivity to triphenyl with respect to benzene:

$$S_{T/B} = \frac{3F_T}{F_{Bf} - F_B}$$

As a consistency check, we recall that the product of fractional conversion and product selectivity (when defined as above) will give the yield to a specific product:

$$Y_{D/B} = X_BS_{D/B} = \frac{F_{Bf} - F_B}{F_{Bf}} \cdot \frac{2F_D}{F_{Bf} - F_B} = \frac{2F_D}{F_{Bf}}$$

$$Y_{T/B} = X_BS_{T/B} = \frac{F_{Bf} - F_B}{F_{Bf}} \cdot \frac{3F_T}{F_{Bf} - F_B} = \frac{3F_T}{F_{Bf}}$$

In [ ]:
## Note that we can still use the function that contains the ODE system in P01(vol, var) above. 
## Nothing about the problem has changed other than how we process the ODE solution.

In [ ]:
## Commented sections of cell here were solved above; I'm adding just to remind us.
## No need to re-solve.

# #This cell solves the ODE system (an initial value problem) using solve_ivp
# FBf     = 60000 #mol/h
# FDf     = 0
# FHf     = 0
# FTf     = 0

# #Solve the problem
# vspan   = (0, 5000)
# var0    = [FBf, FDf, FHf, FTf]
# solP01 = solve_ivp(P01, vspan, var0, atol = 1e-12, rtol = 1e-12)

# #Extract data from the solution structure.

# #Volumes from the ODE solver
# Vout    = solP01.t

# #Molar flowrates as a function of volume.
# FBout   = solP01.y[0]
# FDout   = solP01.y[1]
# FHout   = solP01.y[2]
# FTout   = solP01.y[3]

# #Sum things up to get the total molar flowrate as a function of volume.
# FTot_o  = FBout + FDout + FHout + FTout

# #Calculate mole fractions as a function of volume.
# yBout   = FBout/FTot_o
# yDout   = FDout/FTot_o
# yHout   = FHout/FTot_o
# yTout   = FTout/FTot_o

# #Calculate conversion
# XBout   = (FBf - FBout)/FBf

#Calculate diphenyl/triphenyl yield
YDB     = FDout*2/FBf
YTB     = FTout*3/FBf

#Calculate diphenyl selectivity
SDB     = FDout*2/(FBf - FBout)
STB     = FTout*3/(FBf - FBout)

#Plot yield, selectivity, conversion vs. reactor volume.
plt.figure(figsize = (5, 5))
plt.title('Conversion and Diphenyl Yield in a PFR', fontsize = 14)
plt.plot(Vout, XBout, label = 'Conversion')
plt.plot(Vout, YDB, label = 'Yield to D')
plt.xlim(0, max(vspan))
plt.ylim(0, 1)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('Benzene Conversion/Diphenyl Yield', fontsize = 14)
plt.legend()
plt.show()

plt.figure(figsize = (5, 5))
plt.title('Diphenyl and Triphenyl Selectivity', fontsize = 14)
plt.plot(XBout, SDB, label = 'Selectivity to D')
plt.plot(XBout, STB, label = 'Selectivity to T')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('Benzene Conversion', fontsize = 14)
plt.ylabel('Product Selectivity', fontsize = 14)
plt.legend()
plt.show()

plt.figure(figsize = (5, 5))
plt.title('Diphenyl and Triphenyl Yield', fontsize = 14)
plt.plot(XBout, YDB, label = 'Yield to D')
plt.plot(XBout, YTB, label = 'Yield to T')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('Benzene Conversion', fontsize = 14)
plt.ylabel('Product Yield', fontsize = 14)
plt.legend()
plt.show()

### Finding the value and location of the maximum diphenyl yield

First, we'll use `max()` and `np.argmax()` to get a crude approximation of the maximum, 

```{warning}
This is a relatively crude optimization because we are finding the maximum mole fraction of diphenyl that ***the solver stepped to*** when solving the ODE system.  The true maximum will almost certaintly be slightly different from this.
```

In [ ]:
YDBmax1 = max(YDB)
print(YDBmax1)
print(f'The maximum diphenyl yield is approximately {YDBmax1:0.3f}. \n')

indexmax = np.argmax(YDB)
print(indexmax)
print(YDB[indexmax])
YDBmax2   = YDB[indexmax]
Vmax2    = Vout[indexmax]
print(f'The maximum diphenyl yield is {YDBmax2:0.3f}, which we obtain at a PFR volume of {Vmax2:0.0f}L. \n')

In [ ]:
#Plot yield, selectivity, conversion vs. reactor volume.
plt.figure(figsize = (5, 5))
plt.title('Conversion and Diphenyl Yield in a PFR', fontsize = 14)
plt.scatter(Vout, YDB, label = 'Yield to D')
plt.xlim(0, max(vspan))
plt.ylim(0, 1)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('Benzene Conversion/Diphenyl Yield', fontsize = 14)
plt.legend()
plt.show()

Next, we'll refine that estimate bit by creating an interpolating polynomial that returns Volume as a function of diphenyl yield.  We'll use that with `opt.minimize_scalar()` to find the location of the optimum.  This is the same strategy we introduced for optimization of a scalar function in **587-R08**.

In [ ]:
YvVol = interp1d(Vout, YDB)#, kind = 'cubic')
print(YvVol(800))
vset = np.linspace(0, 5000, 200)
plt.title('Diphenyl Yield', fontsize = 14)
plt.plot(vset, YvVol(vset), label = 'Yield to D')
plt.xlim(0, 5000)
plt.ylim(-1, 1)
plt.xlabel('Volume (L)', fontsize = 14)
plt.ylabel('Product Yield', fontsize = 14)
plt.legend()
plt.show()

In [ ]:
objtemp = lambda V: -1*YvVol(V)
opt_answer = opt.minimize_scalar(objtemp)
print(opt_answer)
print(f'The maximum yield is {-1*opt_answer.fun:3.3f} at a PFR volume of {opt_answer.x:3.0f}L')